In [28]:
%load_ext autoreload
%autoreload 2
import gc
import re
import pandas as pd
import numpy as np
import os
import sys
HOME = os.path.expanduser("~")
sys.path.append(f'{HOME}/kaggle/data_analysis/library')
import utils
from utils import get_categorical_features, get_numeric_features, reduce_mem_usage
import time

key = 'card_id'
target = 'target'
ignore_list = [key, target, 'merchant_id', 'first_active_month']

#========================================================================
# Keras 
# Corporación Favorita Grocery Sales Forecasting
from datetime import date, timedelta
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.advanced_activations import PReLU
from keras.layers.normalization import BatchNormalization
from keras.layers import LSTM
from keras import callbacks
from keras import optimizers
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import gc

from sklearn.model_selection import train_test_split

from keras import backend as K

def RMSE(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
print("Preparing dataset...")
df_train = utils.read_df_pkl('../input/train*0*')
X_test = utils.read_df_pkl('../input/test*0*')
use_cols = [col for col in df_train.columns if col not in ignore_list]
X_train, X_val = train_test_split(df_train, test_size=0.2)

y_train = X_train[target].values
X_train = X_train[use_cols]
X_test = X_test[use_cols]
y_val = X_val[target].values
X_val = X_val[use_cols]

scaler = StandardScaler()
scaler.fit(pd.concat([X_train, X_val, X_test]))
X_train[:] = scaler.transform(X_train)
X_val[:] = scaler.transform(X_val)
X_test[:] = scaler.transform(X_test)

X_train = X_train.as_matrix()
X_test = X_test.as_matrix()
X_val = X_val.as_matrix()
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))
X_val = X_val.reshape((X_val.shape[0], 1, X_val.shape[1]))

def build_model():
    model = Sequential()
    model.add(LSTM(512, input_shape=(X_train.shape[1],X_train.shape[2])))
    model.add(BatchNormalization())
    model.add(Dropout(.2))

    model.add(Dense(256))
    model.add(PReLU())
    model.add(BatchNormalization())
    model.add(Dropout(.1))

    model.add(Dense(256))
    model.add(PReLU())
    model.add(BatchNormalization())
    model.add(Dropout(.1))

    model.add(Dense(128))
    model.add(PReLU())
    model.add(BatchNormalization())
    model.add(Dropout(.05))

    model.add(Dense(64))
    model.add(PReLU())
    model.add(BatchNormalization())
    model.add(Dropout(.05))

    model.add(Dense(32))
    model.add(PReLU())
    model.add(BatchNormalization())
    model.add(Dropout(.05))

    model.add(Dense(16))
    model.add(PReLU())
    model.add(BatchNormalization())
    model.add(Dropout(.05))

    model.add(Dense(1))

    return model

N_EPOCHS = 2000

val_pred = []
test_pred = []

y = y_train
y_mean = y.mean()
xv = X_val
yv = y_val
model = build_model()
opt = optimizers.Adam(lr=0.001)
model.compile(loss=RMSE, optimizer=opt, metrics=[metric])

callbacks = [
    EarlyStopping(monitor='val_loss', patience=10, verbose=0),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=7, verbose=1, epsilon=1e-4, mode='min')
    ]
model.fit(X_train, y - y_mean, batch_size = 65536, epochs = N_EPOCHS, verbose=2,
           validation_data=(xv,yv-y_mean), callbacks=callbacks )
val_pred.append(model.predict(X_val)+y_mean)
test_pred.append(model.predict(X_test)+y_mean)

err = (y_val - np.array(val_pred).squeeze(axis=2).transpose())
err = err.sum(axis=1)
err = np.sqrt(err.sum() / 16)
print('nwrmsle = {}'.format(err))
#========================================================================

100%|██████████| 3/3 [00:00<00:00, 152.18it/s]

Preparing dataset...


Train on 161533 samples, validate on 40384 samples
Epoch 1/2000


KeyboardInterrupt: 

In [ ]:
#========================================================================
# 
#========================================================================
models = Sequential()
models.add(Dense(output_dim=1024, input_dim=input_d, init='lecun_uniform')) 
models.add(Activation('relu')) 
models.add(BatchNormalization())    
models.add(Dropout(0.5))  
models.add(Dense(512, activation='relu',init='lecun_uniform'))
models.add(Activation('relu')) 
models.add(BatchNormalization())    
models.add(Dropout(0.4))  
models.add(Dense(2, init='lecun_uniform'))
models.add(Activation('softmax'))    
opt = optimizers.Adam(lr=0.01)
models.compile(loss='binary_crossentropy', optimizer=opt)